In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import IsolationForest
from scipy.signal import butter, filtfilt
import pywt
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import IsolationForest
import shap
import os
import glob
from sklearn.base import BaseEstimator, TransformerMixin

In [ ]:
def create_leakage_variable(data, structure):
    if structure == 'A':
        return data['Q1'] + data['Q2'] + data['Q3'] + data['Q4'] - data['Q5']
    elif structure == 'B':
        return data['Q1'] - (data['Q2'] + data['Q3'] + data['Q4'])
    elif structure == 'C':
        return data['Q1'] - (data['Q2'] + data['Q3'] + data['Q4'] + data['Q5'] + data['Q6'] + data['Q7'] + data['Q8'])
    elif structure == 'D':
        return data['Q1'] - (data['Q2'] + data['Q3'] + data['Q4'] + data['Q5'])
    else:
        return None

df_A = pd.read_csv("TRAIN_A.csv")

print(df_A.columns)
print("칼럼 수 : ",len(df_A.columns))
print("차원 : ",df_A.shape)


# 새로운 열 생성: Q5 - (Q1 + Q2 + Q3 + Q4)
df_A['leakage'] = df_A['Q1'] + df_A['Q2'] + df_A['Q3'] + df_A['Q4'] - df_A['Q5']
#df_A['leakage_diff'] = df_A['leakage'].diff()

In [ ]:
class AnomalyDetectionStage1(BaseEstimator, TransformerMixin):
    def __init__(self, contamination=0.03, random_state=42, n_estimators=100):
        self.contamination = contamination
        self.random_state = random_state
        self.n_estimators = n_estimators
        self.iso_forest = None

    def fit(self, X, y=None):
        # Isolation Forest 모델 정의 및 학습
        self.iso_forest = IsolationForest(
            contamination=self.contamination,
            random_state=self.random_state,
            n_estimators=self.n_estimators
        )
        leakage_data = X['leakage'].values.reshape(-1, 1)
        self.iso_forest.fit(leakage_data)
        return self

    def transform(self, X):
        df = X.copy()
        
        # 2. 양수 leakage만 필터링
        positive_mask = df['leakage'] > 0
        df_positive = df[positive_mask].copy()

        # 3. Anomaly 예측
        leakage_data = df['leakage'].values.reshape(-1, 1)
        anomaly_pred = self.iso_forest.predict(leakage_data)
        df['predicted_anomaly'] = np.where(anomaly_pred == -1, 1, 0)

        # predicted_anomaly를 stage1_anomaly로 변경
        df = df.rename(columns={'predicted_anomaly': 'stage1_anomaly'})

        # df_positive에도 stage1_anomaly 열 추가
        df_positive['stage1_anomaly'] = df.loc[df_positive.index, 'stage1_anomaly']

        # 결과를 원본 데이터프레임에 추가
        df['df_positive_index'] = df.index.isin(df_positive.index)

        return df  # 이 줄이 추가되었습니다

# 파이프라인 사용 예시
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('stage1', AnomalyDetectionStage1())
])

# 파이프라인 실행
result = pipeline.fit_transform(df_A)

In [ ]:
class AnomalyDetectionStage2(BaseEstimator, TransformerMixin):
    def __init__(self, contamination=0.1, random_state=42, n_estimators=100):
        self.contamination = contamination
        self.random_state = random_state
        self.n_estimators = n_estimators

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        pressure_cols = [col for col in df.columns if col.startswith('P') and col != 'Pi_flag']
        
        # 전처리 적용
        preprocessed_df = self.preprocessing(df, pressure_cols)
        
        # 2단계 모델 적용
        stage2_results = self.detect_pressure_anomalies_stage2(preprocessed_df, pressure_cols)
        
        # 1단계와 2단계 결과 결합
        final_results = pd.DataFrame(index=df.index)
        final_results['stage1_anomaly'] = df['stage1_anomaly']
        final_results['stage2_anomaly'] = stage2_results['stage2_anomaly']

        # AND 연산으로 두 단계에서 모두 anomaly로 예측된 경우만 선택
        final_results['final_anomaly'] = ((final_results['stage1_anomaly'] == 1) & 
                                         (final_results['stage2_anomaly'] == 1)).astype(int)
        
        # 결과를 원본 데이터프레임에 추가
        df = pd.concat([df, final_results], axis=1)
        
        return df

    def preprocessing(self, df, pressure_cols, ema_span=20):
        preprocessed_features = {}
        
        for col in pressure_cols:
            # 1. EMA로 노이즈 제거
            ema = df[col].ewm(span=ema_span, adjust=False).mean()
            
            # 2. 하락 구간 식별
            gradient = ema.diff()
            drop_mask = gradient < 0
            
            # 3. 하락 구간 특성 추출
            # 하락 기간: 연속된 하락 구간의 길이
            drop_duration = pd.Series(0, index=df.index)
            duration_count = 0
            for i, is_dropping in enumerate(drop_mask):
                if is_dropping:
                    duration_count += 1
                else:
                    duration_count = 0
                drop_duration.iloc[i] = duration_count
            
            # 하락 폭: 시작점 대비 현재 하락량
            drop_magnitude = pd.Series(0.0, index=df.index)
            start_value = ema.iloc[0]
            for i, is_dropping in enumerate(drop_mask):
                if is_dropping:
                    drop_magnitude.iloc[i] = start_value - ema.iloc[i]
                else:
                    start_value = ema.iloc[i]
            
            # 하락 속도: gradient의 크기
            drop_speed = gradient.abs() * drop_mask
            
            # 결과 저장
            preprocessed_features[f'{col}_ema'] = ema
            preprocessed_features[f'{col}_drop_duration'] = drop_duration
            preprocessed_features[f'{col}_drop_magnitude'] = drop_magnitude
            preprocessed_features[f'{col}_drop_speed'] = drop_speed
        
        return pd.DataFrame(preprocessed_features, index=df.index)

    def detect_pressure_anomalies_stage2(self, preprocessed_df, pressure_cols):
        drop_features = []
        for col in pressure_cols:
            drop_features.extend([
                f'{col}_drop_duration',
                f'{col}_drop_magnitude',
                f'{col}_drop_speed'
            ])
        
        features_df = preprocessed_df[drop_features].fillna(0)
        
        iso = IsolationForest(
            contamination=self.contamination,
            random_state=self.random_state,
            n_estimators=self.n_estimators
        )
        
        scaler = StandardScaler()
        X_scaled = scaler.fit_transform(features_df)
        
        predictions = iso.fit_predict(X_scaled)
        anomalies = np.where(predictions == -1, 1, 0)
        
        results = pd.DataFrame(index=preprocessed_df.index)
        results['stage2_anomaly'] = anomalies
        
        return results

In [ ]:
class PIFlagDetectionStage3(BaseEstimator, TransformerMixin):
    def __init__(self, window_size=3, min_duration=25, max_duration=35):
        self.window_size = window_size
        self.min_duration = min_duration
        self.max_duration = max_duration

    def fit(self, X, y=None):
        return self

    def transform(self, X):
        df = X.copy()
        pressure_cols = [col for col in df.columns if col.startswith('P') and col != 'Pi_flag']
        pi_flags = self.detect_pi_flags_stage3(df, df['final_anomaly'], pressure_cols, self.window_size)
        
        # pi_flags만 반환
        return pi_flags

    def find_anomaly_periods(self, anomaly_series):
        periods = []
        current_period = []
        
        for idx, value in anomaly_series.items():
            if value == 1:
                current_period.append(idx)
            elif current_period:
                if self.min_duration <= len(current_period) <= self.max_duration:
                    periods.append(current_period)
                current_period = []
        
        if current_period and self.min_duration <= len(current_period) <= self.max_duration:
            periods.append(current_period)
        
        return periods

    def detect_pi_flags_stage3(self, df_A, final_anomaly, pressure_cols, window_size=3):
        # 전체 기간에 대한 Pi별 독립적 Isolation Forest 적용
        flags = pd.DataFrame(0, index=df_A.index, columns=pressure_cols)
        
        # 1. 각 Pi별 독립적 anomaly 탐지
        for i in range(len(pressure_cols) - window_size + 1):
            current_group = pressure_cols[i:i+window_size]
            
            # 각 그룹별 하락 패턴 감지
            for col in current_group:
                iso = IsolationForest(contamination=0.1, random_state=42)
                scaler = StandardScaler()
                
                # 압력 데이터의 하락 특성 추출
                pressure_data = df_A[col].values.reshape(-1, 1)
                gradient = np.gradient(pressure_data.flatten())
                
                features = np.column_stack([
                    pressure_data,
                    gradient,
                    np.roll(gradient, 1)
                ])
                
                X_scaled = scaler.fit_transform(features)
                predictions = iso.fit_predict(X_scaled)
                flags[col] = np.where(predictions == -1, 1, 0)
        
        # 2. Final anomaly 기간 동안의 연속된 압력계 하락 패턴 확인
        final_flags = pd.DataFrame(0, index=df_A.index, columns=pressure_cols)
        anomaly_periods = self.find_anomaly_periods(final_anomaly)
        
        for period in anomaly_periods:
            # 각 period에서 가장 강한 하락 패턴을 보이는 연속된 압력계 그룹 찾기
            max_drops = 0
            best_group = None
            
            for i in range(len(pressure_cols) - window_size + 1):
                current_group = pressure_cols[i:i+window_size]
                group_flags = flags.loc[period, current_group]
                
                # 그룹의 하락 강도 계산
                drop_strength = group_flags.sum().sum()
                
                if drop_strength > max_drops:
                    max_drops = drop_strength
                    best_group = current_group
            
            # 선택된 그룹에 대해서만 flag 설정
            if best_group is not None:
                final_flags.loc[period, best_group] = 1
        
        return final_flags


In [ ]:
# 파이프라인 사용 예시
from sklearn.pipeline import Pipeline

pipeline = Pipeline([
    ('stage1', AnomalyDetectionStage1()),
    ('stage2', AnomalyDetectionStage2()),
    ('stage3', PIFlagDetectionStage3())
])


In [ ]:
# 파이프라인 실행
result = pipeline.fit_transform(df_A)

In [ ]:
result.head()

In [ ]:
def create_submission(df_A, result):
    # 결과를 저장할 DataFrame 생성
    submission = pd.DataFrame(columns=['ID', 'flag_list'])
    
    # A 구조의 압력계 개수 (P1부터 P26까지)
    pressure_cols = [f'P{i}' for i in range(1, 27)]
    
    # pi_flags에서 Pi_flag가 포함된 컬럼들에 대해서만 이상 여부 확인
    flags = [1 if result[f'P{i}_flag'].any() else 0 for i in range(1, 27)]
    
    # 결과 추가
    submission = pd.DataFrame({
        'ID': ['A'],
        'flag_list': [flags]
    })
    
    return submission

In [ ]:
# 실행
submission = create_submission(df_A, result)

# 결과 저장
submission.to_csv('sample_submission.csv', index=False)

# 결과 확인
print(submission)

In [ ]:
def process_single_file(file_path, structure):
    # 1. 파일 읽기 및 전처리
    df_A = pd.read_csv(file_path)
    df_A['leakage'] = create_leakage_variable(df_A, structure)
    
    # 2. 앙상블 모델 단계별 적용
    pipeline = Pipeline([
        ('stage1', AnomalyDetectionStage1()),
        ('stage2', AnomalyDetectionStage2()),
        ('stage3', PIFlagDetectionStage3())
    ])
    
    # 파이프라인 실행
    pi_flags = pipeline.fit_transform(df_A)
    
    # 3. 파일명 추출
    file_name = os.path.basename(file_path).split('.')[0]
    
    # 4. 결과 생성
    if structure == 'C':
        pressure_cols = [f'P{i}' for i in range(1, 9)]
    else:  # structure == 'D'
        pressure_cols = [f'P{i}' for i in range(1, 6)]
    
    flags = [1 if pi_flags[f'P{i}'].any() else 0 for i in range(1, len(pressure_cols) + 1)]
    
    return pd.DataFrame({
        'ID': [file_name],
        'flag_list': [flags]
    })

In [ ]:
def create_final_submission():
    directory_path1 = "C:/Users/leonk/OneDrive/바탕 화면/open/test/C"
    directory_path2 = "C:/Users/leonk/OneDrive/바탕 화면/open/test/D"
    directories = [directory_path1, directory_path2]
    
    all_submissions = []
    
    for i, directory in enumerate(directories):
        structure = 'C' if i == 0 else 'D'
        csv_files = glob.glob(os.path.join(directory, '*.csv'))
        
        for file in csv_files:
            submission = process_single_file(file, structure)
            all_submissions.append(submission)
    
    if all_submissions:
        final_submission = pd.concat(all_submissions, ignore_index=True)
        final_submission.to_csv('submission.csv', index=False)
        print("Submission file 'submission.csv' has been created.")
        return final_submission
    else:
        print("No CSV files found in the directories.")
        return None

In [ ]:
# 실행
create_final_submission()